In [1]:
import numpy as np
import pandas as pd
import os

os.chdir("/g/data/jr19/rh2942/text-empathy/")
from utils.utils import plot, set_all_seeds
from utils.common import DataModule, Trainer

In [2]:
checkpoint = 'roberta-base'
task = ['empathy', 'wrong_empathy']
# feature_to_tokenise=['demographic_essay', 'article']
# feature_to_tokenise=['demographic', 'essay']
feature_to_tokenise=['demographic_essay']

################# COMBINED TRAIN FILE ##############
# train_file = './data/WS22-WS23-sep-from-aug-train-gpt.tsv' # w/o augmentation
train_file = './data/WS22-WS23-augmented-train-gpt.tsv'

################# WASSA 2022 ####################
# dev_label_crowd = './data/WASSA22/goldstandard_dev_2022.tsv'
# dev_file = './data/WS22-dev-gpt.tsv'
# dev_label_gpt = './data/WS22-dev-gpt.tsv'

################# WASSA 2023 ####################
dev_label_crowd = './data/WASSA23/goldstandard_dev.tsv'
dev_file = './data/WS23-dev-gpt.tsv'
dev_label_gpt = './data/WS23-dev-gpt.tsv'

# At different seed and alpha

In [6]:
seed_range = [0, 42, 100, 999, 1234]
anno_diff_range = np.arange(0, 6.5, 0.5)

In [ ]:
val_results = pd.DataFrame()

for seed in seed_range:

    set_all_seeds(seed)
    
    data_module = DataModule(
        task=task,
        checkpoint=checkpoint,
        batch_size=16,
        feature_to_tokenise=feature_to_tokenise,
        seed=seed
    )
    
    train_loader = data_module.dataloader(file=train_file, send_label=True, shuffle=True)
    dev_loader = data_module.dataloader(file=dev_file, send_label=False, shuffle=False)
    # test_loader = data_module.dataloader(file=test_file, send_label=False, shuffle=False)

    for anno_diff in anno_diff_range:
        trainer = Trainer(
            task=task,
            checkpoint=checkpoint,
            lr=1e-5,
            n_epochs=10,
            train_loader=train_loader,
            dev_loader=dev_loader,
            dev_label_gpt=dev_label_gpt,
            dev_label_crowd=dev_label_crowd,
            device_id=0,
            anno_diff=anno_diff,
            mode=0 # -1: crowd, 1: gpt, 0: crowd-gpt
        )

        save_as_loss = './ws22ckp/loss-llm-roberta-seed-' + str(seed) + '-anno_diff-' + str(anno_diff) + '.pth'
        save_as_pearson = './ws22ckp/pearson-llm-roberta-seed-' + str(seed) + '-anno_diff-' + str(anno_diff) + '.pth'
        
        val_pearson_r = trainer.fit(save_as_loss, save_as_pearson)

        # save as seed in index and anno_diff in columns
        print(f'\n----Seed {seed}, anno_diff {anno_diff}: {val_pearson_r}----\n')
        val_results.loc[seed, anno_diff] = val_pearson_r

    # Saving in each seed to be cautious
    val_results.to_csv('ws22-val_results_diff_seed_anno_diff.tsv', sep='\t')

# Saving at the end
val_results.to_csv('ws22-val_results_diff_seed_anno_diff.tsv', sep='\t')

# For a fixed setting

In [ ]:
seed = 0
anno_diff = 6.0

set_all_seeds(seed)

data_module = DataModule(
    task=task,
    checkpoint=checkpoint,
    batch_size=16,
    feature_to_tokenise=feature_to_tokenise,
    seed=seed
)

train_loader = data_module.dataloader(file=train_file, send_label=True, shuffle=True)
dev_loader = data_module.dataloader(file=dev_file, send_label=False, shuffle=False)

trainer = Trainer(
    task=task,
    checkpoint=checkpoint,
    lr=1e-5,
    n_epochs=10,
    train_loader=train_loader,
    dev_loader=dev_loader,
    dev_label_gpt=dev_label_gpt,
    dev_label_crowd=dev_label_crowd,
    device_id=0,
    anno_diff=anno_diff,
    mode=-1 # -1: crowd, 1: gpt, 0: crowd-gpt
)

trainer.fit(save_as_loss=None, save_as_pearson=None)

Map:   0%|          | 0/5268 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1
Train loss: 8.044336840961918
Pearson r: 0.225
Validation loss: 2.8000993728637695
Best dev set Pearson r: 0.225

Epoch: 2
Train loss: 3.197762204661514
Pearson r: 0.273
Validation loss: 2.6078591346740723
Best dev set Pearson r: 0.273

Epoch: 3


# Test

## WS 23

In [ ]:
seed = 0
anno_diff = 5.5

test_file = './data/PREPROCESSED-WS23-test.tsv'
load_model = './ws23ckp/pearson-llm-roberta-seed-' + str(seed) + '-anno_diff-' + str(anno_diff) + '.pth'

## WS 22

In [39]:
seed = 0
anno_diff = 2.0

test_file = './data/PREPROCESSED-WS22-test.tsv'
load_model = './ws22ckp/pearson-llm-roberta-seed-' + str(seed) + '-anno_diff-' + str(anno_diff) + '.pth'

In [40]:
set_all_seeds(seed)

data_module = DataModule(
    task=task,
    checkpoint=checkpoint,
    batch_size=16,
    feature_to_tokenise=feature_to_tokenise,
    seed=seed
)

print('Working with', test_file)
test_loader = data_module.dataloader(file=test_file, send_label=False, shuffle=False)

trainer = Trainer(
    task=task,
    checkpoint=checkpoint,
    lr=1e-5,
    n_epochs=10,
    train_loader=None,
    dev_loader=None,
    dev_label_gpt=None,
    dev_label_crowd=None,
    device_id=0,
    anno_diff=anno_diff,
    mode=0 # -1: crowd, 1: gpt, 0: crowd-gpt
)

Working with ./data/PREPROCESSED-WS22-test.tsv


Map:   0%|          | 0/525 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
print('Working with', load_model)
pred = trainer.evaluate(dataloader=test_loader, load_model=load_model)
pred_df = pd.DataFrame({'emp': pred, 'dis': pred}) # we're not predicting distress, just aligning with submission system
pred_df.to_csv('./tmp/predictions_EMP.tsv', sep='\t', index=None, header=None)
pred_df

Working with ./ws22ckp/pearson-llm-roberta-seed-0-anno_diff-2.0.pth


,emp,dis
0,5.462105,5.462105
1,4.194427,4.194427
2,3.507026,3.507026
3,4.420381,4.420381
4,3.127812,3.127812
...,...,...
520,1.839970,1.839970
521,3.565697,3.565697
522,5.970722,5.970722
523,5.677538,5.677538


In [42]:
%cd tmp
!zip predictions.zip predictions_EMP.tsv

/g/data/jr19/rh2942/text-empathy/tmp
  adding: predictions_EMP.tsv (deflated 66%)


In [43]:
!rm predictions_EMP.tsv predictions.zip
%cd ../

/g/data/jr19/rh2942/text-empathy
